In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt

def load_data():
    df = pd.read_csv('skempi_v2.csv', delimiter=';') #открываем данные с SKEMPI 2.0
    return df

def preprocess_data(df):
    relevant_columns = ['Mutation(s)_PDB', 'iMutation_Location', 'Protein', 'DDG', 'Temperature', 'pH'] #отбор столбцов
    df = df[relevant_columns].copy()
    df = df.dropna()
    df['ddg_sign'] = (df['DDG'] > 0).astype(int) #обозначаем колонкой знак изменения энергии
    df[['chain', 'original_aa', 'position', 'mutated_aa']] = df['Mutation(s)_PDB'].str.extract(r'([A-Za-z])([A-Z])(\d+)([A-Z])') #Извлекаем признаки из мутаций
    amino_acids = list('ACDEFGHIKLMNPQRSTVWY') # Преобразуем аминокислоты в числовые коды
    aa_to_num = {aa: i for i, aa in enumerate(amino_acids)}
    df['original_aa_code'] = df['original_aa'].map(aa_to_num)
    df['mutated_aa_code'] = df['mutated_aa'].map(aa_to_num)
    features = ['original_aa_code', 'mutated_aa_code', 'position', 'Temperature', 'pH']
    X,y = df[features],df['ddg_sign']
    return X, y

def train_and_evaluate(X, y): # Обучение и оценка модели
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    #scaler = StandardScaler()
    #X_train_scaled = scaler.fit_transform(X_train)
    #X_test_scaled = scaler.transform(X_test)
    model = RandomForestClassifier(n_estimators=100, random_state=42) #выбираем рандомный лес - мощный метод из можества деревьев для базовых задач
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print(accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))

    feature_importances = pd.Series(model.feature_importances_, index=X.columns)
    feature_importances.sort_values(ascending=False).plot(kind='bar')
    plt.title("Важность признаков")
    plt.show()

    return model

def main():
    df = load_data()
    if df is None:
        return
    X, y = preprocess_data(df)
    model = train_and_evaluate(X, y)

if __name__ == "__main__":
    main()

KeyError: "['iMutation_Location', 'Protein', 'DDG', 'pH'] not in index"